In [1]:
import sys
sys.path.append('../../src')

In [2]:
import pandas
import numpy
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, ProcessMode, VaryingProcess
from energiapy.components.material import Material
from energiapy.components.location import Location
from energiapy.components.scenario import Scenario
from energiapy.components.result import Result
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.plot import plot_results, plot_scenario
from energiapy.plot.plot_results import CostY, CostX
from energiapy.model.solve import solve

In [3]:
scales=TemporalScale(discretization_list=[1]) 

In [4]:
Biomass= Resource(name='Biomass', price=36.11, cons_max= 10**10, basis='GJ', label='Biomass') #table 2

GridPower = Resource(name='GridPower', cons_max = 10**10, price=9.72, basis='GJ', label='Grid Electricity')

NG = Resource(name='NG', price= 8.89, cons_max= 10*10, basis='GJ', label='Natural gas')
Power=Resource(name='Power', basis='kW',label='Electrical Power')
Heat=Resource(name='Heat', basis='kW',label='Heat Power')

CO2_Vent = Resource(name='CO2_Vent', sell=True, basis='tons', label='Carbon dioxide')

Solar = Resource(name = 'Solar', cons_max=10**10, basis ='GJ', label ='Solar energy') 
Wind = Resource(name = 'Wind', cons_max=10**10, basis ='GJ', label ='Wind energy')

Lighting = Resource(name='Lighting', demand=True, basis='kW',label='Lighting')
Refrigeration = Resource(name='Refrigeration', demand=True, basis='kW',label='Refrigeration') 
Heating = Resource(name='Heating', demand=True, basis='kW',label='Heating')

In [5]:
ST= Process(name='BM_ST', conversion={Biomass: -1, Power: 277.78*0.68, CO2_Vent:100},capex=250, fopex=15,
            prod_min=100, prod_max=1000000, label='Biomass ST', basis='PJ') #table 3 


CHP= Process(name='NG_CHP', conversion={NG: -1, Power: 277.78*0.44, CO2_Vent: 56, Heat: 277.78*0.28},
             capex=500, fopex=15, prod_max=1000000, prod_min=800, label='Natural Gas CHP', basis='PJ')


PV= Process(name='PV', conversion={Solar: -1, Power: 277.78*0.09}, capex=2000, fopex = 500, 
             prod_max=300, prod_min=10, label='Solar PV',  basis='kW')


WF= Process(name='WF',conversion={Wind: -1, Power: 277.78*0.22}, capex=2000, fopex=1200,
             prod_max=500, prod_min=10, label='Wind Farm', basis='kW') 


Grid = Process(name = 'Grid', conversion = {GridPower:-1, Power:277.78, CO2_Vent:90}, 
               basis = 'PJ', label = 'Grid Electricity', prod_max = 1000)

In [6]:
Refrigerator = Process(name = 'Refrigerator', conversion = {Power:-1, Refrigeration:1*3}, capex = 70, vopex = 4, 
             basis = 'kW', label = 'Refrigerator', prod_max = 1000)

LED = Process(name = 'LED', conversion = {Power:-1, Lighting:1*0.8}, capex = 10, vopex = 1, 
             basis = 'kW', label = 'LED', prod_max = 1000)

Heater = Process(name = 'Heater', conversion = {Heat:-1, Heating:1*0.85}, capex = 30, vopex = 3, 
             basis = 'kW', label = 'Heater', prod_max = 1000)

In [7]:
place = Location(name = 'place', processes = {ST, CHP, PV, WF, Grid, Refrigerator, LED, Heater}, 
                 scales = scales, label = 'College Station')

In [8]:
scenario= Scenario(name='scenario', network=place, 
                   demand= {place:{Heating:100, Lighting:200, Refrigeration:1000}},
                   network_scale_level=0,scheduling_scale_level=0, scales=scales, 
                   label='scenario', demand_scale_level = 0)

In [16]:
lp=formulate(scenario=scenario, constraints=
            {
            Constraints.COST,
            Constraints.PRODUCTION,
            Constraints.RESOURCE_BALANCE,
            Constraints.INVENTORY,
            Constraints.NETWORK,
            Constraints.DEMAND})

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint production mode
constraint inventory balance
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint demand


In [17]:
lp.C.pprint()

C : Resource Consumption
    Size=5, Index=C_index
    Key                       : Lower : Value : Upper : Fixed : Stale : Domain
      ('place', 'Biomass', 0) :     0 :  None :  None : False :  True : NonNegativeReals
    ('place', 'GridPower', 0) :     0 :  None :  None : False :  True : NonNegativeReals
           ('place', 'NG', 0) :     0 :  None :  None : False :  True : NonNegativeReals
        ('place', 'Solar', 0) :     0 :  None :  None : False :  True : NonNegativeReals
         ('place', 'Wind', 0) :     0 :  None :  None : False :  True : NonNegativeReals


In [18]:
from pyomo.environ import Objective

lp.objective = Objective(expr = sum(lp.C['place', resource_, 0] for resource_ in lp.resources_purch))

In [19]:
results=solve(scenario= scenario, instance=lp, solver='gurobi', name='lp')


Set parameter QCPDual to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 148 rows, 143 columns and 291 nonzeros
Model fingerprint: 0xd615be88
Variable types: 135 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [8e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 148 rows and 143 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 2.94704 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.947035247247e+00, best

In [20]:
lp.C.pprint()

C : Resource Consumption
    Size=5, Index=C_index
    Key                       : Lower : Value              : Upper : Fixed : Stale : Domain
      ('place', 'Biomass', 0) :     0 :                0.0 :  None : False : False : NonNegativeReals
    ('place', 'GridPower', 0) :     0 : 1.4344423059741573 :  None : False : False : NonNegativeReals
           ('place', 'NG', 0) :     0 : 1.5125929412732766 :  None : False : False : NonNegativeReals
        ('place', 'Solar', 0) :     0 :                0.0 :  None : False : False : NonNegativeReals
         ('place', 'Wind', 0) :     0 :                0.0 :  None : False : False : NonNegativeReals
